# Import dependencies

In [1]:
import pandas as pd
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import subprocess
from sqlalchemy import create_engine

from config import PGEND_POINT
from config import PGDATABASE_NAME
from config import PGUSER_NAME
from config import PGPASSWORD

# AWS RDS Connection

In [2]:
def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+ PGEND_POINT +" port="+ "5432" +" dbname="+ PGDATABASE_NAME +" user=" + PGUSER_NAME \
                  +" password="+ PGPASSWORD
    
    conn = psycopg2.connect(conn_string)
    print("Connected!")

    # Create a cursor object
    cursor = conn.cursor()
    
    return conn, cursor

In [3]:
conn, cursor = connect()

Connected!


# SQL Queries

### Create earthquakes table

In [4]:
# Creating the earthquakes table
query_earthquakes = sql.SQL("""CREATE TABLE earthquakes (
  latitude DECIMAL,
  longitude DECIMAL,
  magnitude DECIMAL,
  event_date DATE
);""")

In [5]:
# Create a cursor object
cur = conn.cursor()

try:
    # Execute your SQL statements here
    cur.execute(query_earthquakes)

    # If everything is successful, commit the transaction
    conn.commit()
    print("Transaction committed successfully!")
except psycopg2.Error as e:
    # If an error occurs, rollback the transaction
    conn.rollback()
    print("Transaction rolled back due to error:", e)
finally:
    # Close the cursor and connection
    cur.close()

Transaction committed successfully!


### Create injectionVolumes table

In [36]:
# Creating the injectionVolumes table
query_injectionVolumes = sql.SQL("""CREATE TABLE injection_volumes (
  api_number INT,
  surface_longitude DECIMAL,
  surface_latitude DECIMAL,
  injection_date DATE,
  injection_end_date DATE,
  volume_injected_bbls DECIMAL
);""")

In [37]:
# Create a cursor object
cur = conn.cursor()

try:
    # Execute your SQL statements here
    cur.execute(query_injectionVolumes)

    # If everything is successful, commit the transaction
    conn.commit()
    print("Transaction committed successfully!")
except psycopg2.Error as e:
    # If an error occurs, rollback the transaction
    conn.rollback()
    print("Transaction rolled back due to error:", e)
finally:
    # Close the cursor and connection
    cur.close()
   

Transaction committed successfully!


### Create pressureData table

In [10]:
# Creating the pressureData table
query_pressureData = sql.SQL("""CREATE TABLE pressure_data (
  time DATE,
  pressure DECIMAL,
  layer VARCHAR(10),
  longitude DECIMAL,
  latitude DECIMAL
);""")

In [11]:
# Create a cursor object
cur = conn.cursor()

try:
    # Execute your SQL statements here
    cur.execute(query_pressureData)

    # If everything is successful, commit the transaction
    conn.commit()
    print("Transaction committed successfully!")
except psycopg2.Error as e:
    # If an error occurs, rollback the transaction
    conn.rollback()
    print("Transaction rolled back due to error:", e)
finally:
    # Close the cursor and connection
    cur.close()
   

Transaction rolled back due to error: relation "pressure_data" already exists



### Create the query_pressureData_13 table

In [21]:
# Creating the query_pressureData_13 table
query_pressureData_13 = sql.SQL("""CREATE TABLE pressure_data_13 (
  time DATE,
  pressure DECIMAL,
  layer VARCHAR(10),
  longitude DECIMAL,
  latitude DECIMAL,
  delta DECIMAL
);""")

In [22]:
# Create a cursor object
cur = conn.cursor()

try:
    # Execute your SQL statements here
    cur.execute(query_pressureData_13)

    # If everything is successful, commit the transaction
    conn.commit()
    print("Transaction committed successfully!")
except psycopg2.Error as e:
    # If an error occurs, rollback the transaction
    conn.rollback()
    print("Transaction rolled back due to error:", e)
finally:
    # Close the cursor and connection
    cur.close()
   

Transaction rolled back due to error: relation "pressure_data_13" already exists



### Create the query_pressureData_9 table

In [ ]:
# Creating the query_pressureData_9 table
query_pressureData_9 = sql.SQL("""CREATE TABLE pressure_data_9 (
  time DATE,
  pressure DECIMAL,
  layer VARCHAR(10),
  longitude DECIMAL,
  latitude DECIMAL,
  delta DECIMAL
);""")

In [ ]:
# Create a cursor object
cur = conn.cursor()

try:
    # Execute your SQL statements here
    cur.execute(query_pressureData_9)

    # If everything is successful, commit the transaction
    conn.commit()
    print("Transaction committed successfully!")
except psycopg2.Error as e:
    # If an error occurs, rollback the transaction
    conn.rollback()
    print("Transaction rolled back due to error:", e)
finally:
    # Close the cursor and connection
    cur.close()
   

### Create the query_pressureData_11 table

In [ ]:
# Creating the query_pressureData_11 table
query_pressureData_11 = sql.SQL("""CREATE TABLE pressure_data_11 (
  time DATE,
  pressure DECIMAL,
  layer VARCHAR(10),
  longitude DECIMAL,
  latitude DECIMAL,
  delta DECIMAL
);""")

In [ ]:
# Create a cursor object
cur = conn.cursor()

try:
    # Execute your SQL statements here
    cur.execute(query_pressureData_11)

    # If everything is successful, commit the transaction
    conn.commit()
    print("Transaction committed successfully!")
except psycopg2.Error as e:
    # If an error occurs, rollback the transaction
    conn.rollback()
    print("Transaction rolled back due to error:", e)
finally:
    # Close the cursor and connection
    cur.close()
   

### Create the query_pressureData_19 table

In [ ]:
# Creating the pressureData table
query_pressureData_19 = sql.SQL("""CREATE TABLE pressure_data_19 (
  time DATE,
  pressure DECIMAL,
  layer VARCHAR(10),
  longitude DECIMAL,
  latitude DECIMAL,
  delta DECIMAL
);""")

In [ ]:
# Create a cursor object
cur = conn.cursor()

try:
    # Execute your SQL statements here
    cur.execute(query_pressureData_19)

    # If everything is successful, commit the transaction
    conn.commit()
    print("Transaction committed successfully!")
except psycopg2.Error as e:
    # If an error occurs, rollback the transaction
    conn.rollback()
    print("Transaction rolled back due to error:", e)
finally:
    # Close the cursor and connection
    cur.close()

### Create the query_pressureData_revised table

In [31]:
# Creating the pressureData table
query_pressureData_revised = sql.SQL("""CREATE TABLE pressure_data_revised (
  time DATE,
  pressure DECIMAL,
  layer VARCHAR(10),
  longitude DECIMAL,
  latitude DECIMAL,
  delta DECIMAL
);""")

In [32]:
# Create a cursor object
cur = conn.cursor()

try:
    # Execute your SQL statements here
    cur.execute(query_pressureData_revised)

    # If everything is successful, commit the transaction
    conn.commit()
    print("Transaction committed successfully!")
except psycopg2.Error as e:
    # If an error occurs, rollback the transaction
    conn.rollback()
    print("Transaction rolled back due to error:", e)
finally:
    # Close the cursor and connection
    cur.close()
   

Transaction committed successfully!


### Define functions and variables required to load data into the tables

In [6]:
def connect():
    # Set up a connection to the postgres server.
    conn_string = f"postgresql+psycopg2://{PGUSER_NAME}:{PGPASSWORD}@{PGEND_POINT}:5432/{PGDATABASE_NAME}"
    engine = create_engine(conn_string)
    print("Connected to the database!")
    return engine

In [7]:
# Define a function to push DataFrames into PGadmin tables that were created
def load_data_to_table(dataframe, table_name, engine):
    dataframe.to_sql(name=table_name, con=engine, if_exists='append', index=False)
    print(f"Data loaded into {table_name} successfully!")


In [8]:
# Connect to the database
engine = connect()

Connected to the database!


In [20]:
# Create DataFrames from the CSV files
injection_volumes_df = pd.read_csv('injectionVolumes-Data/injectionVolumes.csv')
pressure_data_df = pd.read_csv('pressure-data/Updated_Pressure_Data_with_LatLon.csv')
earthquakes_df = pd.read_csv('earthquake-data/earthquakes.csv')
pressure_data_13_df = pd.read_csv('pressure-data/delta_pressure_layer13.csv')
pressure_data_9_df = pd.read_csv('pressure-data/delta_pressure_layer9.csv')
pressure_data_11_df = pd.read_csv('pressure-data/delta_pressure_layer11.csv')
pressure_data_19_df = pd.read_csv('pressure-data/delta_pressure_layer19.csv')
pressure_data_revised_df = pd.read_csv('pressure-data/delta_pressure_all_layers.csv')


### Load earthquakes data

In [10]:
# Rename earthquakes columns to match the table
earthquakes_df.rename(columns={'Latitude': 'latitude', 'Longitude': 'longitude', 'Magnitude': 'magnitude', 'Event_Date': 'event_date'}, inplace=True)

In [11]:
# Load the earthquakes data
load_data_to_table(earthquakes_df, 'earthquakes', engine)

Data loaded into earthquakes successfully!


### Load injectionVolumes data

In [31]:
# Rename injectionVolumes columns to match the table
injection_volumes_df.rename(columns={'API Number': 'api_number', 'Surface Longitude': 'surface_longitude', 'Surface Latitude': 'surface_latitude', 'Injection Date': 'injection_date', 'Injection End Date': 'injection_end_date',
                                     'Volume Injected: BBLs': 'volume_injected_bbls'}, inplace=True)

In [38]:
# Load the injectionVolumes data
load_data_to_table(injection_volumes_df, 'injection_volumes', engine)

Data loaded into injection_volumes successfully!


### Load pressure data

In [42]:
# Rename pressureData columns to match the table
pressure_data_df.rename(columns={'Time': 'time', 'Pressure': 'pressure', 'Layer': 'layer', 'Longitude': 'longitude', 'Latitude': 'latitude'}, inplace=True)

# Load the pressure data
load_data_to_table(pressure_data_df, 'pressure_data', engine)

Data loaded into pressure_data successfully!


### Load pressure data for layer 13

In [27]:
# Rename pressure_data_13 columns to match the table
pressure_data_13_df.rename(columns={'Time': 'time', 'Pressure': 'pressure', 'Layer': 'layer', 'Longitude': 'longitude', 'Latitude': 'latitude', 'Delta': 'delta'}, inplace=True)

# Load the pressure data
load_data_to_table(pressure_data_13_df, 'pressure_data_13', engine)

Data loaded into pressure_data_13 successfully!


### Load pressure data for layer 9

In [ ]:
# Rename pressure_data_9 columns to match the table
pressure_data_9_df.rename(columns={'Time': 'time', 'Pressure': 'pressure', 'Layer': 'layer', 'Longitude': 'longitude', 'Latitude': 'latitude', 'Delta': 'delta'}, inplace=True)

# Load the pressure data
load_data_to_table(pressure_data_9_df, 'pressure_data_9', engine)

### Load pressure data for layer 11

In [ ]:
# Rename pressure_data_11 columns to match the table
pressure_data_11_df.rename(columns={'Time': 'time', 'Pressure': 'pressure', 'Layer': 'layer', 'Longitude': 'longitude', 'Latitude': 'latitude', 'Delta': 'delta'}, inplace=True)

# Load the pressure data
load_data_to_table(pressure_data_11_df, 'pressure_data_11', engine)

### Load pressure data for layer 19

In [ ]:
# Rename pressure_data_19 columns to match the table
pressure_data_19_df.rename(columns={'Time': 'time', 'Pressure': 'pressure', 'Layer': 'layer', 'Longitude': 'longitude', 'Latitude': 'latitude', 'Delta': 'delta'}, inplace=True)

# Load the pressure data
load_data_to_table(pressure_data_19_df, 'pressure_data_19', engine)

### Load pressure data for all layers

In [37]:
# Rename pressure_data_revised columns to match the table
pressure_data_revised_df.rename(columns={'Time': 'time', 'Pressure': 'pressure', 'Layer': 'layer', 'Longitude': 'longitude', 'Latitude': 'latitude', 'Delta': 'delta'}, inplace=True)

# Load the pressure data
load_data_to_table(pressure_data_revised_df, 'pressure_data_revised', engine)

Data loaded into pressure_data_revised successfully!
